# setup

In [ ]:
#load libraries
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx


import urllib.request as url
import glob
import csv

from glambie_demo_notebooks.helpers import glambie_regions_dropdown, glambie_years_dropdown, derivative_to_cumulative, load_all_region_dataframes
from glambie_demo_notebooks.plotting import global_cumulative_plot, global_comparison_stacked_region_plot, histogram_of_region_contributions_to_global_loss_two_years, histogram_of_region_contributions_to_global_loss, histogram_of_region_contributions_to_global_loss_two_years, global_stacked_all_regions_plot
from glambie_demo_notebooks.plotting import single_region_derivative_plot, single_region_cumulative_plot, two_region_comparison_plot, transform_string
from glambie.const import regions
from glambie.data import timeseries

sns.set_theme(palette='colorblind', style='white', font_scale=1.2)
sns.set_style({'axes.linewidth': 2, 'axes.edgecolor':'lightgrey'})

colors_list = sns.color_palette(palette='colorblind', n_colors=18, desat=0.9)

In [ ]:
input_dir = Path("input")
output_dir = Path("output")
input_rel = input_dir / "glambie_reference"
input_maps = input_dir / "maps"
input_datasets = input_dir / "input_datasets"
glambie_runs = input_dir / "glambie_runs"
output_sensitivity = output_dir / "sensitivity"
output_rel = output_dir / "relative_change"
output_dir_datasets = output_dir / "datasets"

# functions

## data

In [ ]:
def difference_between_runs_plot(data_run_1, data_run_2):
    #merge 
    merged_df = data_run_1.merge(data_run_2, on= ('start_dates', 'end_dates', 'region'),suffixes=('_1', '_2'), copy = False)

    #calculate difference
    merged_df['gt_difference'] = merged_df.combined_gt_2 - merged_df.combined_gt_1
    merged_df['gt_errors_difference'] = merged_df.combined_gt_errors_2 - merged_df.combined_gt_errors_1

    merged_df['mwe_difference'] = merged_df.combined_mwe_2 - merged_df.combined_mwe_1
    merged_df['mwe_errors_difference'] = np.abs(merged_df.combined_mwe_errors_2 - merged_df.combined_mwe_errors_1)

    return merged_df

Test code

In [ ]:
glambie_runs

In [ ]:
list(glambie_runs.glob("datasets_default*/0_*/consensus/csvs/global_gt.csv"))

In [ ]:
default_path = glambie_runs / "datasets_default/0_global/consensus/csvs/global_gt.csv"
default_run = pd.read_csv(default_path)

including_most_path = glambie_runs / "datasets_including_most/0_global/consensus/csvs/global_gt.csv"
including_most_run = pd.read_csv(including_most_path)

## plots

### two runs

In [ ]:
def two_runs_comparison_plot(region_name, data_distinction, comparison_data_distinction, data_run_1, data_run_2):
    
    _, axs = plt.subplots(1, 2, figsize=(20,8))

    #plot 1 = gt changes timeseries 
    axs[0].set_xlim(2000,2024)

    axs[0].hlines(0, 2000, 2024, linewidth=2, colors=['grey'], linestyle='dashed')

    #dataset 1
    axs[0].plot(data_run_1.start_dates + 0.5, data_run_1.combined_gt, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction)
    axs[0].fill_between(data_run_1.start_dates + 0.5, data_run_1.combined_gt - data_run_1.combined_gt_errors,
                        data_run_1.combined_gt + data_run_1.combined_gt_errors, alpha=0.2)

    #dataset 2
    axs[0].plot(data_run_2.start_dates + 0.5, data_run_2.combined_gt, linewidth=3, zorder=2, label=region_name + ' - ' + comparison_data_distinction)
    axs[0].fill_between(data_run_2.start_dates + 0.5, data_run_2.combined_gt - data_run_2.combined_gt_errors,
                        data_run_2.combined_gt + data_run_2.combined_gt_errors, alpha=0.2)

    axs[0].set_xlabel('Year')
    axs[0].set_ylabel('Mass Change (Gt)')
    axs[0].grid(True, alpha=0.8)

    ###########3
    #plot 2 = mwe changes timeseries
    axs[1].set_xlim(2000,2024)

    axs[1].hlines(0, 2000, 2024, linewidth=2, colors=['grey'], linestyle='dashed')

    #dataset 1
    axs[1].plot(data_run_1.start_dates + 0.5, data_run_1.combined_mwe, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction)
    axs[1].fill_between(data_run_1.start_dates + 0.5, data_run_1.combined_mwe - data_run_1.combined_mwe_errors,
                        data_run_1.combined_mwe + data_run_1.combined_mwe_errors, alpha=0.2)

    axs[1].plot(data_run_2.start_dates + 0.5, data_run_2.combined_mwe, linewidth=3, zorder=2, label=region_name + ' - ' + comparison_data_distinction)
    axs[1].fill_between(data_run_2.start_dates + 0.5, data_run_2.combined_mwe - data_run_2.combined_mwe_errors,
                        data_run_2.combined_mwe + data_run_2.combined_mwe_errors, alpha=0.2)

    axs[1].set_xlabel('Year')
    axs[1].set_ylabel('Mass Change (mwe)')

    axs[0].legend(loc = 'lower left', fontsize=16)
    plt.suptitle(region_name + ': comparison ' + data_distinction + ' and ' + comparison_data_distinction + ': total ice loss, 2000 - 2024', fontsize=18)
    axs[1].grid(True, alpha=0.8)


    return

In [ ]:
two_runs_comparison_plot

In [ ]:
def two_runs_cumulative_comparison_plot(region_name, data_distinction, comparison_data_distinction, cumulative_data_all_gt, cumulative_errors_all_gt, cumulative_data_all_gt_comparison,
                               cumulative_errors_all_gt_comparison, cumulative_data_all_mwe, cumulative_errors_all_mwe,
                               cumulative_data_all_mwe_comparison, cumulative_errors_all_mwe_comparison):
    
    _, axs = plt.subplots(1, 2, figsize=(20,8))

    #plot 1 
    axs[0].set_xlim(2000,2024)
    axs[0].hlines(0, 2000, 2024, linewidth=2, colors=['grey'], linestyle='dashed')

    axs[0].plot(cumulative_data_all_gt.dates, cumulative_data_all_gt.changes, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction)
    axs[0].fill_between(cumulative_data_all_gt.dates, cumulative_data_all_gt.changes - cumulative_errors_all_gt.errors,
                        cumulative_data_all_gt.changes + cumulative_errors_all_gt.errors, alpha=0.2)

    axs[0].plot(cumulative_data_all_gt_comparison.dates, cumulative_data_all_gt_comparison.changes, linewidth=3, zorder=2, label=region_name + ' - ' + comparison_data_distinction)
    axs[0].fill_between(cumulative_data_all_gt_comparison.dates, cumulative_data_all_gt_comparison.changes - cumulative_errors_all_gt_comparison.errors,
                        cumulative_data_all_gt_comparison.changes + cumulative_errors_all_gt_comparison.errors, alpha=0.2)

    axs[0].set_xlabel('Year')
    axs[0].set_ylabel('Cumulative Change [Gt]')
    axs[0].grid(True, alpha=0.8)

    #plot 2
    axs[1].set_xlim(2000,2024)
    axs[1].hlines(0, 2000, 2024, linewidth=2, colors=['grey'], linestyle='dashed')

    axs[1].plot(cumulative_data_all_mwe.dates, cumulative_data_all_mwe.changes, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction)
    axs[1].fill_between(cumulative_data_all_mwe.dates, cumulative_data_all_mwe.changes - cumulative_errors_all_mwe.errors,
                        cumulative_data_all_mwe.changes + cumulative_errors_all_mwe.errors, alpha=0.2)

    axs[1].plot(cumulative_data_all_mwe_comparison.dates, cumulative_data_all_mwe_comparison.changes, linewidth=3, zorder=2, label=region_name + ' - ' + comparison_data_distinction)
    axs[1].fill_between(cumulative_data_all_mwe_comparison.dates, cumulative_data_all_mwe_comparison.changes - cumulative_errors_all_mwe_comparison.errors,
                        cumulative_data_all_mwe_comparison.changes + cumulative_errors_all_mwe_comparison.errors, alpha=0.2)

    axs[1].set_xlabel('Year')
    axs[1].set_ylabel('Cumulative Change [metres water equivalent]')

    axs[0].legend(loc = 'lower left', fontsize=16)
    plt.suptitle(region_name + ': comparison ' + data_distinction + ' and ' + comparison_data_distinction + ': total ice loss, 2000 - 2023', fontsize=18)
    axs[1].grid(True, alpha=0.8)


    return

In [ ]:
def two_runs_difference_plot(region_name, data_distinction, comparison_data_distinction, difference_data):
    
    _, axs = plt.subplots(1, 2, figsize=(20,8))

    #plot 1 = gt changes timeseries 
    axs[0].set_xlim(2000,2024)

    axs[0].hlines(0, 2000, 2024, linewidth=2, colors=['grey'], linestyle='dashed')
    axs[0].plot(difference_data.start_dates + 0.5, difference_data.gt_difference, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction)

    axs[0].set_xlabel('Year')
    axs[0].set_ylabel('Difference in Mass Change (Gt)')
    axs[0].grid(True, alpha=0.8)

    ###########3
    #plot 2 = mwe changes timeseries
    axs[1].set_xlim(2000,2024)

    axs[1].hlines(0, 2000, 2024, linewidth=2, colors=['grey'], linestyle='dashed')
    axs[1].plot(difference_data.start_dates + 0.5, difference_data.mwe_difference, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction)

    axs[1].set_xlabel('Year')
    axs[1].set_ylabel('Difference in Mass Change (mwe)')

    axs[0].legend(loc = 'lower left', fontsize=16)
    plt.suptitle(region_name + ': Difference in ice loss ' + data_distinction + ' and ' + comparison_data_distinction, fontsize=18)
    axs[1].grid(True, alpha=0.8)


    return

### three runs

In [ ]:
def three_runs_comparison_plot(region_name, data_distinction_1, data_distinction_2, data_distinction_3, data_run_1, data_run_2, data_run_3):
    
    _, axs = plt.subplots(1, 2, figsize=(20,8))

    #plot 1 = gt changes timeseries 
    axs[0].set_xlim(2000,2024)

    axs[0].hlines(0, 2000, 2024, linewidth=2, colors=['grey'], linestyle='dashed')

    #dataset 1
    axs[0].plot(data_run_1.start_dates + 0.5, data_run_1.combined_gt, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction_1)
    axs[0].fill_between(data_run_1.start_dates + 0.5, data_run_1.combined_gt - data_run_1.combined_gt_errors,
                        data_run_1.combined_gt + data_run_1.combined_gt_errors, alpha=0.2)

    #dataset 2
    axs[0].plot(data_run_2.start_dates + 0.5, data_run_2.combined_gt, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction_2)
    axs[0].fill_between(data_run_2.start_dates + 0.5, data_run_2.combined_gt - data_run_2.combined_gt_errors,
                        data_run_2.combined_gt + data_run_2.combined_gt_errors, alpha=0.2)

    #dataset 3
    axs[0].plot(data_run_3.start_dates + 0.5, data_run_3.combined_gt, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction_3)
    axs[0].fill_between(data_run_3.start_dates + 0.5, data_run_3.combined_gt - data_run_3.combined_gt_errors,
                        data_run_3.combined_gt + data_run_3.combined_gt_errors, alpha=0.2)

    axs[0].set_xlabel('Year')
    axs[0].set_ylabel('Mass Change (Gt)')
    axs[0].grid(True, alpha=0.8)

    ###########3
    #plot 2 = mwe changes timeseries
    axs[1].set_xlim(2000,2024)

    axs[1].hlines(0, 2000, 2024, linewidth=2, colors=['grey'], linestyle='dashed')

    #dataset 1
    axs[1].plot(data_run_1.start_dates + 0.5, data_run_1.combined_mwe, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction_1)
    axs[1].fill_between(data_run_1.start_dates + 0.5, data_run_1.combined_mwe - data_run_1.combined_mwe_errors,
                        data_run_1.combined_mwe + data_run_1.combined_mwe_errors, alpha=0.2)

    axs[1].plot(data_run_2.start_dates + 0.5, data_run_2.combined_mwe, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction_2)
    axs[1].fill_between(data_run_2.start_dates + 0.5, data_run_2.combined_mwe - data_run_2.combined_mwe_errors,
                        data_run_2.combined_mwe + data_run_2.combined_mwe_errors, alpha=0.2)

    axs[1].plot(data_run_3.start_dates + 0.5, data_run_3.combined_mwe, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction_3)
    axs[1].fill_between(data_run_3.start_dates + 0.5, data_run_3.combined_mwe - data_run_3.combined_mwe_errors,
                        data_run_3.combined_mwe + data_run_3.combined_mwe_errors, alpha=0.2)

    axs[1].set_xlabel('Year')
    axs[1].set_ylabel('Mass Change (mwe)')

    axs[0].legend(loc = 'lower left', fontsize=16)
    plt.suptitle(region_name + ': comparison ' + data_distinction_1 + ', ' + data_distinction_2 + ' and ' + data_distinction_3 + ': total ice loss, 2000 - 2024', fontsize=18)
    axs[1].grid(True, alpha=0.8)


    return

In [ ]:
def three_runs_difference_plot(region_name, data_distinction_1, data_distinction_2, difference_data_1, difference_data_2):
    
    _, axs = plt.subplots(1, 2, figsize=(20,8))

    #plot 1 = gt changes timeseries 
    axs[0].set_xlim(2000,2024)

    axs[0].hlines(0, 2000, 2024, linewidth=2, colors=['grey'], linestyle='dashed')
    axs[0].plot(difference_data_1.start_dates + 0.5, difference_data_1.gt_difference, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction_1)
    axs[0].plot(difference_data_2.start_dates + 0.5, difference_data_2.gt_difference, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction_2)


    axs[0].set_xlabel('Year')
    axs[0].set_ylabel('Difference in Mass Change (Gt)')
    axs[0].grid(True, alpha=0.8)

    ###########3
    #plot 2 = mwe changes timeseries
    axs[1].set_xlim(2000,2024)

    axs[1].hlines(0, 2000, 2024, linewidth=2, colors=['grey'], linestyle='dashed')
    axs[1].plot(difference_data_1.start_dates + 0.5, difference_data_1.mwe_difference, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction_1)
    axs[1].plot(difference_data_2.start_dates + 0.5, difference_data_2.mwe_difference, linewidth=3, zorder=2, label=region_name + ' - ' + data_distinction_2)

    axs[1].set_xlabel('Year')
    axs[1].set_ylabel('Difference in Mass Change (mwe)')

    axs[0].legend(loc = 'lower left', fontsize=16)
    plt.suptitle(region_name + ': Difference in ice loss between runs', fontsize=18)
    axs[1].grid(True, alpha=0.8)


    return

# Analysis

## Look at area shape

In [ ]:
#load global datasets
global_normal = pd.read_csv(os.path.join(d_dir_normal, '0_global.csv')).drop('Unnamed: 0', axis=1)
global_2012 = pd.read_csv(os.path.join(d_dir_2012, '0_global.csv')).drop('Unnamed: 0', axis=1)
global_2023 = pd.read_csv(os.path.join(d_dir_2023, '0_global.csv')).drop('Unnamed: 0', axis=1)

global_normal['app'] = 705644/ global_normal.glacier_area * global_normal.combined_mwe

#tryout = timeseries.Timeseries.apply_or_remove_area_change(al_normal,6, apply_area_change=True)
#tryout

global_normal.head()
#global_2023.head()

In [ ]:
plt.plot(global_normal.start_dates, global_normal.combined_mwe, color='blue' )
plt.plot(global_normal.start_dates, global_normal.app, color='green')

## 2 runs difference

### changes in GT and area

In [ ]:
for i in range(0,len(list_of_folders)):
    #define region name and path to results
    region = list_of_folders[i]
    
    #get the region name without  the number
    region_name = region[region.find("_")+1:-4]

    #read in the data
    region_normal_df = pd.read_csv(os.path.join(d_dir_normal, region)).drop('Unnamed: 0', axis=1)
    region_comparison_df = pd.read_csv(os.path.join(d_dir_2012, region)).drop('Unnamed: 0', axis=1)

    #plot
    two_runs_comparison_plot(region_name, 'Normal', '2012', region_normal_df, region_comparison_df)


In [ ]:
for i in range(0,len(list_of_folders)):
    #define region name and path to results
    region = list_of_folders[i]
    
    #get the region name without  the number
    region_name = region[region.find("_")+1:-4]

    #read in the data
    region_normal_df = pd.read_csv(os.path.join(d_dir_normal, region)).drop('Unnamed: 0', axis=1)
    region_comparison_df = pd.read_csv(os.path.join(d_dir_2023, region)).drop('Unnamed: 0', axis=1)

    #plot
    two_runs_comparison_plot(region_name, 'Normal', '2023', region_normal_df, region_comparison_df)

### cumulative changes in GT and area

In [ ]:
for i in range(0,len(list_of_folders)):
    #define region name and path to results
    region = list_of_folders[i]
    
    #get the region name without  the number
    region_name = region[region.find("_")+1:-4]

    #read in the data
    region_normal = pd.read_csv(os.path.join(d_dir_normal, region)).drop('Unnamed: 0', axis=1)
    region_2012 = pd.read_csv(os.path.join(d_dir_2012, region)).drop('Unnamed: 0', axis=1)

    #calculate cumulative for normal
    cum_region_gt_6 = derivative_to_cumulative(region_normal.start_dates, region_normal.end_dates, region_normal.combined_gt)
    cum_region_gt_err_6 = derivative_to_cumulative(region_normal.start_dates, region_normal.end_dates, region_normal.combined_gt_errors, calculate_as_errors=True)

    cum_region_mwe_6 = derivative_to_cumulative(region_normal.start_dates, region_normal.end_dates, region_normal.combined_mwe)
    cum_region_mwe_err_6 = derivative_to_cumulative(region_normal.start_dates, region_normal.end_dates, region_normal.combined_mwe_errors, calculate_as_errors=True)

    #calculate cumulative for 2012
    cum_region_gt_7 = derivative_to_cumulative(region_2012.start_dates, region_2012.end_dates, region_2012.combined_gt)
    cum_region_gt_err_7 = derivative_to_cumulative(region_2012.start_dates, region_2012.end_dates, region_2012.combined_gt_errors, calculate_as_errors=True)

    cum_region_mwe_7 = derivative_to_cumulative(region_2012.start_dates, region_2012.end_dates, region_2012.combined_mwe)
    cum_region_mwe_err_7 = derivative_to_cumulative(region_2012.start_dates, region_2012.end_dates, region_2012.combined_mwe_errors, calculate_as_errors=True)

    #plot
    two_runs_cumulative_comparison_plot(region_name, 'normal', '2012', cum_region_gt_6, cum_region_gt_err_6, cum_region_gt_7,
                           cum_region_gt_err_7, cum_region_mwe_6, cum_region_mwe_err_6,
                           cum_region_mwe_7, cum_region_mwe_err_7)

    #save to folder
    #plt.savefig('C:/Users/Milva/Documents/Uni Freiburg/Semester 2.1/capstone/RGI6 vs 7 plots/' + region_name + '_comparison.png')

  

In [ ]:
for i in range(0,len(list_of_folders)):
    #define region name and path to results
    region = list_of_folders[i]
    
    #get the region name without  the number
    region_name = region[region.find("_")+1:-4]

    #read in the data
    region_normal = pd.read_csv(os.path.join(d_dir_normal, region)).drop('Unnamed: 0', axis=1)
    region_2023 = pd.read_csv(os.path.join(d_dir_2023, region)).drop('Unnamed: 0', axis=1)

    #calculate cumulative for normal
    cum_region_gt_6 = derivative_to_cumulative(region_normal.start_dates, region_normal.end_dates, region_normal.combined_gt)
    cum_region_gt_err_6 = derivative_to_cumulative(region_normal.start_dates, region_normal.end_dates, region_normal.combined_gt_errors, calculate_as_errors=True)

    cum_region_mwe_6 = derivative_to_cumulative(region_normal.start_dates, region_normal.end_dates, region_normal.combined_mwe)
    cum_region_mwe_err_6 = derivative_to_cumulative(region_normal.start_dates, region_normal.end_dates, region_normal.combined_mwe_errors, calculate_as_errors=True)

    #calculate cumulative for 2012
    cum_region_gt_7 = derivative_to_cumulative(region_2023.start_dates, region_2023.end_dates, region_2023.combined_gt)
    cum_region_gt_err_7 = derivative_to_cumulative(region_2023.start_dates, region_2023.end_dates, region_2023.combined_gt_errors, calculate_as_errors=True)

    cum_region_mwe_7 = derivative_to_cumulative(region_2023.start_dates, region_2023.end_dates, region_2023.combined_mwe)
    cum_region_mwe_err_7 = derivative_to_cumulative(region_2023.start_dates, region_2023.end_dates, region_2023.combined_mwe_errors, calculate_as_errors=True)

    #plot
    two_runs_cumulative_comparison_plot(region_name, 'normal', '2023', cum_region_gt_6, cum_region_gt_err_6, cum_region_gt_7,
                           cum_region_gt_err_7, cum_region_mwe_6, cum_region_mwe_err_6,
                           cum_region_mwe_7, cum_region_mwe_err_7)

### difference between runs

In [ ]:
for i in range(0,len(list_of_folders)):
    #define region name and path to results
    region = list_of_folders[i]
    
    #get the region name without  the number
    region_name = region[region.find("_")+1:-4]

    #read in the data
    region_normal_df = pd.read_csv(os.path.join(d_dir_normal, region)).drop('Unnamed: 0', axis=1)
    region_comparison_df = pd.read_csv(os.path.join(d_dir_2012, region)).drop('Unnamed: 0', axis=1)

    #plot
    two_runs_difference_plot(region_name, 'Normal', '2012', difference_between_runs_plot(region_normal_df, region_comparison_df))

In [ ]:
for i in range(0,len(list_of_folders)):
    #define region name and path to results
    region = list_of_folders[i]
    
    #get the region name without  the number
    region_name = region[region.find("_")+1:-4]

    #read in the data
    region_normal_df = pd.read_csv(os.path.join(d_dir_normal, region)).drop('Unnamed: 0', axis=1)
    region_comparison_df = pd.read_csv(os.path.join(d_dir_2023, region)).drop('Unnamed: 0', axis=1)

    #plot
    two_runs_difference_plot(region_name, 'Normal', '2023', difference_between_runs_plot(region_normal_df, region_comparison_df))

## 3 runs difference

### changes in GT and area

In [ ]:
for i in range(0,len(list_of_folders)):
    #define region name and path to results
    region = list_of_folders[i]
    
    #get the region name without  the number
    region_name = region[region.find("_")+1:-4]

    #read in the data
    region_normal_df = pd.read_csv(os.path.join(d_dir_normal, region)).drop('Unnamed: 0', axis=1)
    region_2012_df = pd.read_csv(os.path.join(d_dir_2012, region)).drop('Unnamed: 0', axis=1)
    region_2023_df = pd.read_csv(os.path.join(d_dir_2023, region)).drop('Unnamed: 0', axis=1)


    #plot
    three_runs_comparison_plot(region_name, 'Normal', '2012', '2023', region_normal_df, region_2012_df, region_2023_df)


### difference between runs

In [ ]:
for i in range(0,len(list_of_folders)):
    #define region name and path to results
    region = list_of_folders[i]
    
    #get the region name without  the number
    region_name = region[region.find("_")+1:-4]

    #read in the data
    region_normal_df = pd.read_csv(os.path.join(d_dir_normal, region)).drop('Unnamed: 0', axis=1)
    region_2012_df = pd.read_csv(os.path.join(d_dir_2012, region)).drop('Unnamed: 0', axis=1)
    region_2023_df = pd.read_csv(os.path.join(d_dir_2023, region)).drop('Unnamed: 0', axis=1)

    #calculate difference dfs
    nor_12_diff = difference_between_runs_plot(region_normal_df, region_2012_df)
    nor_23_diff = difference_between_runs_plot(region_normal_df, region_2023_df)

    #plot
    three_runs_difference_plot(region_name, 'Normal - 2012', 'Normal - 2023', nor_12_diff, nor_23_diff)